In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from models.galata import Galata
import json

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Galata(device=device)

In [ ]:
model.load_state_dict(torch.load('.checkpoints/best/best_kostantiniyye_thickness_final.pth'), strict = False)

In [ ]:
model.softness.load_state_dict(torch.load('.checkpoints/best/best_kostantiniyye_softness_final.pth'), strict = False)
model.smoothness.load_state_dict(torch.load('.checkpoints/best/best_kostantiniyye_smoothness_final.pth'), strict = False)
model.thickness.load_state_dict(torch.load('.checkpoints/best/best_kostantiniyye_thickness_final.pth'), strict = False)
model.flexibility.load_state_dict(torch.load('.checkpoints/best/best_kostantiniyye_flexibility_final.pth'), strict = False)


In [ ]:
torch.save('galata_final.pth')

In [ ]:
model.to(device)

In [ ]:
with open('data/json/softness_test.json', 'r') as f:
    file = json.load(f)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/csv/touch.csv')

In [ ]:
from imagebind.data import load_and_transform_vision_data

In [ ]:
for item in file:
    im = item['img']
    po = torch.Tensor([item['portion']]).to(device)
    la = item['label']
    re = model(im, po)

In [ ]:
to = { i['id'] : [i['부드러움'], i['매끄러운'], i['두꺼운'], i['신축성']] for i in df.to_dict(orient='records')}

In [ ]:
im, po, la = [],[],[]
for item in file:
    im.append(load_and_transform_vision_data([item['img']], device=device))
    po.append(torch.Tensor([item['portion']]))
    la.append(to[item['id']])



In [ ]:
dataset = torch.utils.data.TensorDataset(torch.cat(im).to('cpu'),torch.cat(po).to('cpu'),torch.Tensor(la).to('cpu') )

In [ ]:
torch.save(dataset, 'test_constantinople.pth')

In [ ]:
len(file)*32

In [ ]:
with torch.no_grad():
    model.eval()
    for item in file:
        im = item['img']
        po = torch.Tensor([item['portion']]).to(device)
        la = item['label']
        re = model(im, po)
        print(la-1, re)